# Lesson 2: Class Warfare

> Disclaimer: Most of these points should only be applied to Python.

- Overutilising or underutilising classes can lead to ruin
- Classes can be a powerful tool or an endless garden path

## Benefits vs Drawbacks

### Benefits

- Can keep track of state
  - No need to pass parameters back and forth
  - No thread-unsafe global variables
  - Can logically initialise state and then use it
- Can organise a hierarcy of states that belong together
- Provide dot-methods for accessing properties
  - "ask, don't tell"

- We have a collection of files
- Must get some attributes from each, and add those into a shared collection

## Functions vs Classes

### Functions vs Methods

In python

- a **function** takes parameters, returns a value
- a **method** can be called on an object, and can access state in the object

## An Example

- You have a collection of items, in this case ids and emails
- Need to iterate through them, collect some values, and pass them on

In [1]:
from faker import Faker
from utils import ppj
fake = Faker()

def fake_record(i, spanner=False):
    if spanner and i % 5 == 0:
        return (i, fake.uuid4(), fake.email(), None)
    else:
        return (i, fake.uuid4(), fake.email(), fake.pyint())
    
for i in range(10):
    collection = [fake_record(i) for i in range(10)]
collection[:-1][-1] = False
ppj(collection[:2])

def iterate(c):
    '''
    This method will yield a tuple of each item, and a boolean indicating 
    if there are more items.
    After all items are consumed, this method will yield None/False
    (This means that we can't just do "for item in iterate(collection)")
    '''
    for el in c[:-2]:
        yield el, True
    yield c[-1], False
    while True:
        yield None, False

[
  [
    0,
    "90c5a78b-5a98-412b-ac38-75c9237bdcc6",
    "lisa92@acosta.net",
    4385
  ],
  [
    1,
    "5d619a00-e2c5-4f94-92ca-6320155e5287",
    "calvarado@yahoo.com",
    6823
  ]
]



_For our exercise, we are only going to collect the ID if the last column has the value "true"_

In [2]:
ids = []
for el, has_next in iterate(collection):
    print(el, has_next)
    if el[3] % 2:
        ids.append(el[1])
    if not has_next:
        break

(0, '90c5a78b-5a98-412b-ac38-75c9237bdcc6', 'lisa92@acosta.net', 4385) True
(1, '5d619a00-e2c5-4f94-92ca-6320155e5287', 'calvarado@yahoo.com', 6823) True
(2, 'ee44850f-9b5e-4c4a-aa6e-f00690d0bd94', 'kevin38@hotmail.com', 6622) True
(3, '0d215529-e561-4b8d-92bd-31a57f1c5a39', 'fwalsh@hotmail.com', 1132) True
(4, 'ff41defb-e6ac-4c65-81ec-f3b2ec460543', 'fmunoz@yahoo.com', 4544) True
(5, '58badd05-a8c2-4f1f-80c7-706132651a2f', 'amberbrown@gmail.com', 6959) True
(6, 'd29e5b85-24cc-42c1-80af-4c4b243feddb', 'phylliskim@fisher-norman.com', 4374) True
(7, '45eaaa48-62ba-43a5-adeb-eb1f9a0dc555', 'mackenzie42@gaines.com', 4794) True
(9, '95dc1f89-0287-42ba-8989-14cb50c1058d', 'thomasnoble@gmail.com', 9781) False


### Adding more stuff

Let's add some details around how many items we consumed/are up to

In [3]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        print(i, el, has_next)
        if el[3] % 2:
            ids.append(el[1])
        if has_next == False:
            break

## Problems

Let's add a spanner

In [15]:
collection = [fake_record(i, True) for i in range(10)]
collection[:-1][-1] = False

process(collection)

0 (0, 'b81afb04-8ace-401f-9fed-e9d0f8124c19', 'steven91@long.com', None) True
1 (1, 'c2702775-a8c9-4a89-94e0-da0aba80b4bf', 'jeffreywilson@cline.org', 790) True
!! 1-c2702775-a8c9-4a89-94e0-da0aba80b4bf-jeffreywilson@cline.org-790
2 (2, '6138615b-66a8-49d6-8f1a-9fd6543a6ada', 'brandondennis@yahoo.com', 9504) True
!! 2-6138615b-66a8-49d6-8f1a-9fd6543a6ada-brandondennis@yahoo.com-9504
3 (3, '4fc5da45-f309-4c33-8d07-b773f2f414e8', 'olivia64@gmail.com', 4207) True
4 (4, '10b4aeac-7177-486d-9f03-224d78fe8b22', 'karen37@baker.com', 6609) True
5 (5, 'f3f644b8-d5b4-41c4-917f-c1f5c324b876', 'jeffrey63@gmail.com', None) True
6 (6, 'a4c8ba4c-eddd-4db5-9d99-80269bb29776', 'david28@yahoo.com', 8843) True
7 (7, '4543ac48-f772-4752-8b6f-0c1c0fdb97fb', 'victoriarichardson@gmail.com', 4045) True
!! 7-4543ac48-f772-4752-8b6f-0c1c0fdb97fb-victoriarichardson@gmail.com-4045
8 (9, '911b500b-dffc-4a20-8556-767190d6aa1e', 'mark31@gmail.com', 1103) False


In [16]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        print(i, el, has_next)
        if isinstance(el, int) and el[3] % 2:
            ids.append(el[1])
        if len(el[2]) > 15:
            print(f'{"-".join(el)}')
        if has_next == False:
            break

In [17]:
process(collection)

0 (0, 'b81afb04-8ace-401f-9fed-e9d0f8124c19', 'steven91@long.com', None) True


TypeError: sequence item 0: expected str instance, int found

In [18]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        print(i, el, has_next)
        if isinstance(el, int) and el[3] % 2:
            ids.append(el[1])
        if len(el[2]) > 20:
            print(f'!! {"-".join(map(str, el))}')
        if has_next == False:
            break
process(collection)

0 (0, 'b81afb04-8ace-401f-9fed-e9d0f8124c19', 'steven91@long.com', None) True
1 (1, 'c2702775-a8c9-4a89-94e0-da0aba80b4bf', 'jeffreywilson@cline.org', 790) True
!! 1-c2702775-a8c9-4a89-94e0-da0aba80b4bf-jeffreywilson@cline.org-790
2 (2, '6138615b-66a8-49d6-8f1a-9fd6543a6ada', 'brandondennis@yahoo.com', 9504) True
!! 2-6138615b-66a8-49d6-8f1a-9fd6543a6ada-brandondennis@yahoo.com-9504
3 (3, '4fc5da45-f309-4c33-8d07-b773f2f414e8', 'olivia64@gmail.com', 4207) True
4 (4, '10b4aeac-7177-486d-9f03-224d78fe8b22', 'karen37@baker.com', 6609) True
5 (5, 'f3f644b8-d5b4-41c4-917f-c1f5c324b876', 'jeffrey63@gmail.com', None) True
6 (6, 'a4c8ba4c-eddd-4db5-9d99-80269bb29776', 'david28@yahoo.com', 8843) True
7 (7, '4543ac48-f772-4752-8b6f-0c1c0fdb97fb', 'victoriarichardson@gmail.com', 4045) True
!! 7-4543ac48-f772-4752-8b6f-0c1c0fdb97fb-victoriarichardson@gmail.com-4045
8 (9, '911b500b-dffc-4a20-8556-767190d6aa1e', 'mark31@gmail.com', 1103) False


In [19]:
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        print(i, el, has_next)
        # collect item if it's even
        if isinstance(el, int) and el[3] % 2:
            ids.append(el[1])
        # Warn about large items
        if len(el[2]) > 20:
            print(f'!! {"-".join(map(str, el))}')
        if has_next == False:
            break

process(collection)

0 (0, 'b81afb04-8ace-401f-9fed-e9d0f8124c19', 'steven91@long.com', None) True
1 (1, 'c2702775-a8c9-4a89-94e0-da0aba80b4bf', 'jeffreywilson@cline.org', 790) True
!! 1-c2702775-a8c9-4a89-94e0-da0aba80b4bf-jeffreywilson@cline.org-790
2 (2, '6138615b-66a8-49d6-8f1a-9fd6543a6ada', 'brandondennis@yahoo.com', 9504) True
!! 2-6138615b-66a8-49d6-8f1a-9fd6543a6ada-brandondennis@yahoo.com-9504
3 (3, '4fc5da45-f309-4c33-8d07-b773f2f414e8', 'olivia64@gmail.com', 4207) True
4 (4, '10b4aeac-7177-486d-9f03-224d78fe8b22', 'karen37@baker.com', 6609) True
5 (5, 'f3f644b8-d5b4-41c4-917f-c1f5c324b876', 'jeffrey63@gmail.com', None) True
6 (6, 'a4c8ba4c-eddd-4db5-9d99-80269bb29776', 'david28@yahoo.com', 8843) True
7 (7, '4543ac48-f772-4752-8b6f-0c1c0fdb97fb', 'victoriarichardson@gmail.com', 4045) True
!! 7-4543ac48-f772-4752-8b6f-0c1c0fdb97fb-victoriarichardson@gmail.com-4045
8 (9, '911b500b-dffc-4a20-8556-767190d6aa1e', 'mark31@gmail.com', 1103) False


### Let's take a step back

We use values from the raw item without knowing that they're usable

Instead of holding all the logic in this method, what if we could _ask_ each element if it was even?

In [20]:
from dataclasses import asdict, dataclass

@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int
        
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        el = Element(*el)
        print(i, el, has_next)
        # collect item if it's even
        if isinstance(el.score, int) and el.score % 2:
            ids.append(el.uuid)
        # Warn about large items
        if len(el.email) > 20:
            print(f'!! {"-".join(map(str, asdict(el).values()))}')
        if has_next == False:
            break
process(collection)

0 Element(numeric_id=0, uuid='b81afb04-8ace-401f-9fed-e9d0f8124c19', email='steven91@long.com', score=None) True
1 Element(numeric_id=1, uuid='c2702775-a8c9-4a89-94e0-da0aba80b4bf', email='jeffreywilson@cline.org', score=790) True
!! 1-c2702775-a8c9-4a89-94e0-da0aba80b4bf-jeffreywilson@cline.org-790
2 Element(numeric_id=2, uuid='6138615b-66a8-49d6-8f1a-9fd6543a6ada', email='brandondennis@yahoo.com', score=9504) True
!! 2-6138615b-66a8-49d6-8f1a-9fd6543a6ada-brandondennis@yahoo.com-9504
3 Element(numeric_id=3, uuid='4fc5da45-f309-4c33-8d07-b773f2f414e8', email='olivia64@gmail.com', score=4207) True
4 Element(numeric_id=4, uuid='10b4aeac-7177-486d-9f03-224d78fe8b22', email='karen37@baker.com', score=6609) True
5 Element(numeric_id=5, uuid='f3f644b8-d5b4-41c4-917f-c1f5c324b876', email='jeffrey63@gmail.com', score=None) True
6 Element(numeric_id=6, uuid='a4c8ba4c-eddd-4db5-9d99-80269bb29776', email='david28@yahoo.com', score=8843) True
7 Element(numeric_id=7, uuid='4543ac48-f772-4752-8b6f-

In [10]:
from dataclasses import asdict, dataclass

@dataclass
class Element:
    numeric_id: int
    uuid: str
    email: str
    score: int

    def is_even(self) -> bool:
        try:
            return self.score % 2
        except TypeError:
            return False

    def email_too_long(self, limit=20) -> bool:
        return len(self.email) > limit
    
    def as_row(self, delim='-'):
        return delim.join(map(str, [
            self.numeric_id, self.uuid, self.email, self.score,
        ]))
        
        
def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        el = Element(*el)
        print(i, el, has_next)

        if el.is_even:
            ids.append(el.uuid)
        if el.email_too_long:
            print(f'!! {el.as_row()}')

        if has_next == False:
            break
process(collection)

0 Element(numeric_id=0, uuid='c874d923-4b38-4b16-9007-b68aa8ede91a', email='lwest@yahoo.com', score=None) True
!! 0-c874d923-4b38-4b16-9007-b68aa8ede91a-lwest@yahoo.com-None
1 Element(numeric_id=1, uuid='8e8500ca-b3d1-4673-9f27-df3b2b4607a2', email='daltontoni@burke.com', score=4724) True
!! 1-8e8500ca-b3d1-4673-9f27-df3b2b4607a2-daltontoni@burke.com-4724
2 Element(numeric_id=2, uuid='5631c54f-998f-4797-af81-b3f2a6f05973', email='samuellynch@wright.com', score=3975) True
!! 2-5631c54f-998f-4797-af81-b3f2a6f05973-samuellynch@wright.com-3975
3 Element(numeric_id=3, uuid='deff8f82-379d-4758-bd76-0a728e204de2', email='bishopchristine@gray.com', score=3006) True
!! 3-deff8f82-379d-4758-bd76-0a728e204de2-bishopchristine@gray.com-3006
4 Element(numeric_id=4, uuid='cabc3165-ef4e-4d29-b479-2698f9137ec7', email='hhenderson@hotmail.com', score=8951) True
!! 4-cabc3165-ef4e-4d29-b479-2698f9137ec7-hhenderson@hotmail.com-8951
5 Element(numeric_id=5, uuid='47dc445f-0dfc-43b0-8a15-7545c548a2fd', email

In [11]:
from typing import List
from dataclasses import field

@dataclass
class ElementCollection:
    items: List[Element] = field(default_factory=list) 

    def from_raw(collection):
        return ElementCollection([Element(*el) for el in collection])
    
    def even_items(self):
        for el in self.items:
            if el.is_even():
                yield el

c = ElementCollection.from_raw(collection)
evens = list(c.even_items())
print(len(evens), len(collection))

4 10


## Now, let's look at all the code together again

In [12]:
from typing import List
from dataclasses import field

@dataclass
class ElementCollection:
    items: List[Element] = field(default_factory=list) 

    def from_raw(collection):
        return ElementCollection([Element(*el) for el in collection])
    
    def even_items(self):
        for el in self.items:
            if el.is_even():
                yield el

def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        print(i, el, has_next)
        if isinstance(el, int) and el[3] % 2:
            ids.append(el[1])
        if len(el[2]) > 20:
            print(f'!! {"-".join(map(str, el))}')
        if has_next == False:
            break

process(collection)

0 (0, 'c874d923-4b38-4b16-9007-b68aa8ede91a', 'lwest@yahoo.com', None) True
1 (1, '8e8500ca-b3d1-4673-9f27-df3b2b4607a2', 'daltontoni@burke.com', 4724) True
2 (2, '5631c54f-998f-4797-af81-b3f2a6f05973', 'samuellynch@wright.com', 3975) True
!! 2-5631c54f-998f-4797-af81-b3f2a6f05973-samuellynch@wright.com-3975
3 (3, 'deff8f82-379d-4758-bd76-0a728e204de2', 'bishopchristine@gray.com', 3006) True
!! 3-deff8f82-379d-4758-bd76-0a728e204de2-bishopchristine@gray.com-3006
4 (4, 'cabc3165-ef4e-4d29-b479-2698f9137ec7', 'hhenderson@hotmail.com', 8951) True
!! 4-cabc3165-ef4e-4d29-b479-2698f9137ec7-hhenderson@hotmail.com-8951
5 (5, '47dc445f-0dfc-43b0-8a15-7545c548a2fd', 'jonathan76@waller.com', None) True
!! 5-47dc445f-0dfc-43b0-8a15-7545c548a2fd-jonathan76@waller.com-None
6 (6, 'f4d436e8-10aa-407e-a08c-99ee98912516', 'amorales@hotmail.com', 137) True
7 (7, 'e8597e3a-a05e-430a-a533-b1952adab147', 'wmichael@hotmail.com', 4110) True
8 (9, '1b135dfe-4f80-4fdb-b107-2be73b9a7e07', 'monica84@johnson.net'

In [26]:
from dataclasses import field

@dataclass
class Iterator:
    raw: list
    klass: type
    
    def __iter__(self):
        for el, has_next in iterate(self.raw):
            yield self.klass(*el)
            if not has_next:
                return

def process(collection):
    ids = []
    for i, (el, has_next) in enumerate(iterate(collection)):
        print(i, el, has_next)
        if isinstance(el, int) and el[3] % 2:
            ids.append(el[1])
        if len(el[2]) > 20:
            print(f'!! {"-".join(map(str, el))}')
        if has_next == False:
            break

for el in Iterator(collection, Element):
    print(el)

ElementCollection(items=[Element(numeric_id=0, uuid='b81afb04-8ace-401f-9fed-e9d0f8124c19', email='steven91@long.com', score=None), Element(numeric_id=1, uuid='c2702775-a8c9-4a89-94e0-da0aba80b4bf', email='jeffreywilson@cline.org', score=790), Element(numeric_id=2, uuid='6138615b-66a8-49d6-8f1a-9fd6543a6ada', email='brandondennis@yahoo.com', score=9504), Element(numeric_id=3, uuid='4fc5da45-f309-4c33-8d07-b773f2f414e8', email='olivia64@gmail.com', score=4207), Element(numeric_id=4, uuid='10b4aeac-7177-486d-9f03-224d78fe8b22', email='karen37@baker.com', score=6609), Element(numeric_id=5, uuid='f3f644b8-d5b4-41c4-917f-c1f5c324b876', email='jeffrey63@gmail.com', score=None), Element(numeric_id=6, uuid='a4c8ba4c-eddd-4db5-9d99-80269bb29776', email='david28@yahoo.com', score=8843), Element(numeric_id=7, uuid='4543ac48-f772-4752-8b6f-0c1c0fdb97fb', email='victoriarichardson@gmail.com', score=4045), Element(numeric_id=9, uuid='911b500b-dffc-4a20-8556-767190d6aa1e', email='mark31@gmail.com', s